# SAR Calibration Tool: Display results included in calibration database

The current Jupyter Notebook can be used to display the results generated by the SAR Calibration Tool and stored in the calibration database.

## Setup

Setup required environment.

Note that, after the setup of the SCT Python environment, the following commands shall be manually executed in order to install the modules required by the current Jupyter Notebook to run:

        conda install notebook
        conda install matplotlib
        conda install seaborn

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Define useful inputs, required to locate and identify the calibration database:

In [ ]:
calibration_site_id = 'surat_basin'
calibration_db_file = os.path.join('../../resources/calibration_db/', calibration_site_id, 'calibration_db__2020.xlsx')

Set other useful parameters:

In [ ]:
# Figures properties
sns.set_style('whitegrid', rc={'grid.linestyle': ':'})
sns.set_context('notebook', font_scale=1.5)
marker_size = 200
qualitative_palette = 'Set1'   # qualitative_palette = sns.set_palette(sns.color_palette(["#FF0B04", ...]))
sequential_palette = 'mako'
ale_rg_min = -0.5   # [m]
ale_rg_max =  0.5   # [m]
ale_az_min = -2     # [m]
ale_az_max =  2     # [m]
save_figures = False   # NOTE Requires write permissions in notebook folder

# Useful variables
calibration_site_tag = calibration_site_id.replace('_',' ').title()
mission_list = ['S1A', 'S1B']
swath_list   = ['IW1', 'IW2', 'IW3']
orbit_direction_polarization_dict = {'ASCENDING H/H': '2', 'ASCENDING V/V': '4', 'DESCENDING H/H': '1', 'DESCENDING V/V': '3'}
time_ref = datetime(year=2020, month=1, day=1, hour=0, minute=0, second=0)

Note: the save_figures variable can be used to save figures as .png files.

## Load calibration database

Load calibration database as a Pandas dataframe:

In [ ]:
df = pd.read_excel(calibration_db_file)

# df.info()
# df.head()

Derive additional useful information and add them to the dataframe:

In [ ]:
df['Mission'] = df['Product name'].str.slice(start=0, stop=3)

df['Orbit direction + Polarization'] = df['Orbit direction']+' '+df['Polarization']

time_start = df['Product name'].str.slice(start=17, stop=32).apply(lambda x: datetime.strptime(x, "%Y%m%dT%H%M%S"))
time_stop  = df['Product name'].str.slice(start=33, stop=48).apply(lambda x: datetime.strptime(x, "%Y%m%dT%H%M%S"))
df['Time [UTC]'] = time_start+(time_stop-time_start)/2
df['Time [days]'] = (df['Time [UTC]']-time_ref).apply(lambda x: x.total_seconds()/60/60/24)

df['Range ALE [m]'] = df['Measured range ALE [m]'] - df['Doppler shift [m]'] - df['Ionospheric delay [m]'] \
                      - df['Tropospheric delay (h) [m]'] - df['Tropospheric delay (w) [m]']
df['Azimuth ALE [m]'] = df['Measured azimuth ALE [m]'] - df['Bistatic delay [m]'] - df['Instrument timing [m]'] - df['FM rate shift [m]']
df['Overall ALE [m]'] = np.int32(np.rint(np.linalg.norm(df[['Range ALE [m]','Azimuth ALE [m]']], axis=1)))

valid_flag = (df['SCR [dB]']>=20) & (df['Overall ALE [m]']<=10)   # reliable measurements, i.e. SCR>=20dB and ALE<=10m

## Display results

### SCR

Display Signal-to-Clutter Ratio versus Target ID:

In [ ]:
fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(12,6))

sns.scatterplot(x='Target ID', y='SCR [dB]', data=df, ax=ax, \
                hue='Overall ALE [m]', palette=sequential_palette, alpha=0.75, \
                style='Orbit direction + Polarization', style_order=orbit_direction_polarization_dict.keys(), \
                markers=orbit_direction_polarization_dict, s=marker_size)

ax.axhline(y=20, color='k', alpha=0.25)
ax.legend(loc='center left', bbox_to_anchor=(1,0.5), markerscale=2)

if save_figures:
    fig.savefig(calibration_site_tag+' - SCR vs Target ID vs ALE.png', bbox_inches='tight')

### Range vs Azimuth ALE

Display range and azimuth ALE statistics per mission:

In [ ]:
print('\tRange ALE [m]\t\tAzimuth ALE [m]')
for m in range(len(mission_list)):
    data = df[(df['Mission']==mission_list[m]) & (valid_flag==True)]
    print('{}\t{:+.3f} +/- {:+.3f}\t{:+.3f} +/- {:+.3f}'.format(mission_list[m], \
        data['Range ALE [m]'].mean(), data['Range ALE [m]'].std(), data['Azimuth ALE [m]'].mean(), data['Azimuth ALE [m]'].std()))

Display range and azimuth ALE statistics per mission and swath:

In [ ]:
print('\t\tRange ALE [m]\t\tAzimuth ALE [m]')
for m in range(len(mission_list)):
    for s in range(len(swath_list)):
        data = df[(df['Mission']==mission_list[m]) & (df['Swath']==swath_list[s]) & (valid_flag==True)]
        print('{} {}\t\t{:+.3f} +/- {:+.3f}\t{:+.3f} +/- {:+.3f}'.format(mission_list[m], swath_list[s], \
            data['Range ALE [m]'].mean(), data['Range ALE [m]'].std(), data['Azimuth ALE [m]'].mean(), data['Azimuth ALE [m]'].std()))

Display range versus azimuth ALE per mission:

In [ ]:
fig,ax = plt.subplots(nrows=1, ncols=len(mission_list), figsize=(20,10))
for m in range(len(mission_list)):
    sns.scatterplot(x='Range ALE [m]', y='Azimuth ALE [m]', data=df[(df['Mission']==mission_list[m]) & (valid_flag==True)], ax=ax[m], \
                    hue='Swath', hue_order=swath_list, palette=qualitative_palette, alpha=0.75, \
                    style='Orbit direction + Polarization', style_order=orbit_direction_polarization_dict.keys(), \
                    markers=orbit_direction_polarization_dict, s=marker_size)
    ax[m].axvline(x=0, color='k', alpha=0.25)
    ax[m].axhline(y=0, color='k', alpha=0.25)
    ax[m].set_aspect('equal')
    ax[m].set_xlim(ale_rg_min, ale_rg_max)
    ax[m].set_ylim(ale_az_min, ale_az_max)
    ax[m].set_xlabel('Slant range offset [m]')
    ax[m].set_ylabel('Azimuth offset [m]')
    ax[m].set_title(calibration_site_tag+'\n'+mission_list[m]+' Azimuth vs Range ALE [m]', weight='bold')
    ax[m].legend(loc='center left', bbox_to_anchor=(1,0.5), markerscale=2)

if save_figures:
    fig.savefig(calibration_site_tag+' - Azimuth vs Range ALE [m].png', bbox_inches='tight')

Display range versus azimuth ALE per mission and with swath histograms:

In [ ]:
for m in range(len(mission_list)):
    
    df_curr = df[(df['Mission']==mission_list[m]) & (valid_flag==True)]
    
    g = sns.JointGrid(xlim=(ale_az_min, ale_az_max), ylim=(ale_az_min, ale_az_max), height=10)
    # note: same axes limits to avoid using g.ax_joint.set_aspect('equal'), that creates issues with marginal axes height
    g.ax_joint.axvline(x=0, color='k', alpha=0.25)
    g.ax_joint.axhline(y=0, color='k', alpha=0.25)
    
    sns.scatterplot(x="Range ALE [m]", y="Azimuth ALE [m]", data=df_curr, ax=g.ax_joint, \
                    hue='Swath', hue_order=swath_list, palette=qualitative_palette, alpha=0.75, \
                    style='Orbit direction + Polarization', style_order=orbit_direction_polarization_dict.keys(), \
                    markers=orbit_direction_polarization_dict, s=marker_size)
    
    sns.kdeplot(x="Range ALE [m]", data=df_curr, ax=g.ax_marg_x, 
                hue='Swath', hue_order=swath_list, palette=qualitative_palette, legend=False)
    
    sns.kdeplot(y="Azimuth ALE [m]", data=df_curr, ax=g.ax_marg_y, 
                hue='Swath', hue_order=swath_list, palette=qualitative_palette, legend=False)
    
    g.ax_joint.set_xlabel('Slant range offset [m]')
    g.ax_joint.set_ylabel('Azimuth offset [m]')
    g.ax_marg_x.set_title(calibration_site_tag+'\n'+mission_list[m]+' Azimuth vs Range ALE [m]', weight='bold')
    g.ax_joint.legend(loc='center left', bbox_to_anchor=(1.25,0.5), markerscale=2)
    
    if save_figures:
        g.savefig(calibration_site_tag+' - '+mission_list[m]+' Azimuth vs Range ALE [m].png', bbox_inches='tight')

Display range versus azimuth ALE 2D histograms per mission and swath:

In [ ]:
g = sns.FacetGrid(data=df[valid_flag==True], row='Swath', row_order=swath_list, col='Mission', col_order=mission_list, \
                  xlim=(ale_rg_min,ale_rg_max), ylim=(ale_az_min,ale_az_max))
g.map(plt.axvline, x=0, color='k', alpha=0.25)
g.map(plt.axhline, y=0, color='k', alpha=0.25)
g.map(sns.kdeplot, 'Range ALE [m]', 'Azimuth ALE [m]', cmap=sequential_palette+'_r', fill='true')

if save_figures:
    g.savefig(calibration_site_tag+' - Azimuth vs Range ALE [m] (2D histograms).png', bbox_inches='tight')

### Range and Azimuth ALE vs various quantities

Display range and azimuth ALE versus various quantities, i.e.:
- Incidence angle
- Squint angle
- Time
- Target ID


In [ ]:
def get_plot_details(x):
    plot_details_dict = {
        'ALE vs Time':            {'xAxis': 'Time [days]',           'xAxisTag': 'Time (relative) [days]', 'xLim': (0,np.ceil(max(df['Time [days]'])))}, 
        'ALE vs Target ID':       {'xAxis': 'Target ID',             'xAxisTag': 'PT ID []',               'xLim': (min(df['Target ID'])-1,max(df['Target ID'])+1)   }, 
        'ALE vs Incidence angle': {'xAxis': 'Incidence angle [deg]', 'xAxisTag': 'Incidence angle [deg]',  'xLim': (30,46),                            }, 
        'ALE vs Squint angle':    {'xAxis': 'Squint angle [deg]',    'xAxisTag': 'Squint angle [deg]',     'xLim': (-0.5,0.5),                         }, 
    }[x]
    return plot_details_dict['xAxis'], plot_details_dict['xAxisTag'], plot_details_dict['xLim'], ['Range ALE [m]','Azimuth ALE [m]'], ['Slant range offset [m]','Azimuth offset [m]'], [(ale_rg_min,ale_rg_max),(ale_az_min,ale_az_max)]

plot_tag_list = ['ALE vs Incidence angle', 'ALE vs Squint angle', 'ALE vs Time', 'ALE vs Target ID']
for p in plot_tag_list:
    
    x_axis, x_axis_tag, x_lim, y_axis, y_axis_tag, y_lim = get_plot_details(p)
    
    for v in range(len(y_axis)):
        
        g = sns.FacetGrid(data=df[valid_flag==True], col='Mission', col_order=mission_list, \
                          hue='Swath', hue_order=swath_list, palette=qualitative_palette, \
                          height=4, aspect=2, xlim=x_lim, ylim=y_lim[v])
        
        g.map(plt.axhline, y=0, color='k', alpha=0.25)
        
        g.map(sns.scatterplot, x_axis, y_axis[v], style=df[valid_flag==True]['Orbit direction + Polarization'], alpha=0.75, \
              style_order=orbit_direction_polarization_dict.keys(), markers=orbit_direction_polarization_dict, s=marker_size)
        
        g.set_axis_labels(x_axis_tag, y_axis_tag[v])
    
        if save_figures:
            g.savefig(calibration_site_tag+' - '+p+' ('+str(v)+').png', bbox_inches='tight')